Zadanie polega na przewidzeniu, czy roczny dochód danej osoby przekracza 50 000 USD (zmienna powyżej lub poniżej progu „$50K/yr”) na podstawie danych demograficznych i społeczno-zawodowych pochodzących z bazy spisu ludności.

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
# metadata 
#print(adult.metadata) 
  
# variable information 
print(adult.variables) 


Liczba rekordów: 48842.

Liczba atrybutów: 14 cech wejściowych (predyktorów) oraz 1 zmienna celu.

Charakterystyka danych: Zbiór zawiera zarówno zmienne numeryczne (całkowitoliczbowe), jak i kategoryczne (tekstowe).

Zmienna celu (income): Zmienna binarna przyjmująca wartości >50K lub<=50K.

In [ ]:
# Pierwsze 5 wierszy cech
print(X.head())

# Informacje o typach danych i liczbie niepustych wartości
print(X.info())

1. **age:** Wiek badanej osoby (liczba ciągła)
2. **workclass:** Forma zatrudnienia (np. Private, Self-emp-not-inc, Federal-gov)
3. **fnlwgt:** Waga demograficzna, liczba osób reprezentowanych przez rekord. (final weight)
4. **education:** Najwyższy stopień edukacji (np. Bachelors, Masters, HS-grad)
5. **education-num:** Poziom edukacji ujęty numerycznie (liczba ciągła)
6. **marital-status:** Stan cywilny (np. Married-civ-spouse, Never-married)
7. **occupation:** Wykonywany zawód (np. Tech-support, Exec-managerial)
8. **relationship:** Rola w rodzinie (np. Husband, Not-in-family)
9. **race:** Rasa (np. White, Black, Asian-Pac-Islander)
10. **sex:** Płeć (Male, Female)
11. **capital-gain:** Zyski kapitałowe (liczba ciągła)
12. **capital-loss:** Straty kapitałowe (liczba ciągła)
13. **hours-per-week:** Liczba przepracowanych godzin w tygodniu (liczba ciągła)
14. **native-country:** Kraj pochodzenia (np. United-States, Mexico, Germany)

In [ ]:
# Unikalne wartości w kolumnie celu
print(y.value_counts())

In [ ]:
# Sprawdzenie czy w danych występują znaki zapytania
print((X == '?').sum())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Kopia danych, aby nie modyfikować oryginału z repozytorium
df_X = X.copy()
df_y = y.copy()

# Stan przed zmianami
print(f"Liczba wierszy przed czyszczeniem: {len(df_X)}")

Najpierw oczyszczono zmienną income, bo w danych były kropki na końcu niektórych kategorii. Usunięto je i zamieniono tekst na wartości binarne, gdzie zero oznacza zarobki do 50 tysięcy, a jedynka powyżej.

In [ ]:
# Czyszczenie zmiennej celu
# Usuwamy kropki, które pojawiają się w niektórych rekordach
df_y['income'] = df_y['income'].astype(str).str.replace('.', '', regex=False)

# Mapowanie na wartości binarne (0 i 1)
# <=50K -> 0
# >50K -> 1
df_y['income'] = df_y['income'].map({'<=50K': 0, '>50K': 1})

print("Rozkład klas po czyszczeniu:")
print(df_y['income'].value_counts())

W zbiorze Adult braki danych są ukryte pod znakami zapytania, co jest problematyczne dla algorytmów. Zamieniono wszystkie te znaki na format zrozumiały dla Pythona (NaN), a potem użyliśmy strategii SimpleImputer, wstawiając w puste miejsca najczęściej występujące wartości z danej kolumny.

In [ ]:
# Zamiana '?' na np.nan
df_X.replace('?', np.nan, inplace=True)

# Sprawdzenie ile mamy teraz prawdziwych braków
print("\nLiczba braków danych w kolumnach:")
print(df_X.isnull().sum())

# Imputacja braków danych najczęstszą wartością
# Tworzymy imputer
imputer = SimpleImputer(strategy='most_frequent')

# Uzupełniamy dane
df_X_imputed = pd.DataFrame(imputer.fit_transform(df_X), columns=df_X.columns)

# Przywracamy poprawne typy danych
df_X_imputed = df_X_imputed.infer_objects()

# Upewnienie że kolumny numeryczne są numeryczne
num_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
for col in num_cols:
    df_X_imputed[col] = df_X_imputed[col].astype(int)

Zdecydowano usunąć dwie kolumny, czyli education oraz fnlwgt. Pierwsza jest zbędna, bo mamy jej numeryczny odpowiednik, a druga to waga demograficzna, która przy przewidywaniu zarobków konkretnej osoby mogłaby wprowadzić szum.

In [ ]:
# Usuwanie kolumn
# 'education' jest zduplikowana informacją z 'education-num'
# 'fnlwgt' to waga demograficzna, zazwyczaj nieużyteczna w predykcji dochodu
cols_to_drop = ['education', 'fnlwgt']
df_X_cleaned = df_X_imputed.drop(columns=cols_to_drop)

print(f"\nKolumny po usunięciu zbędnych: {df_X_cleaned.columns.tolist()}")

W celu poprawy jakości danych uzyto funkcję usuwającą wartości odstające metodą rozstępu międzykwartylowego dla wybranych cech numerycznych, takich jak wiek czy czas pracy. Proces ten pozwolił na oczyszczenie zbioru z szumu statystycznego, przy jednoczesnym zachowaniu zmiennych finansowych, w których skrajne wartości niosą kluczową informację dla predykcji wysokich dochodów

In [ ]:
# Usuwanie Outlierów

def remove_outliers_iqr(df_features, df_target, columns_to_filter, factor=1.5):
    
    # Tworzenie kopii
    X_temp = df_features.copy()
    y_temp = df_target.copy()
    
    # Początkowo zachowujemy wszystkie indeksy
    valid_indices = X_temp.index

    for col in columns_to_filter:
        Q1 = X_temp[col].quantile(0.25)
        Q3 = X_temp[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - (factor * IQR)
        upper_bound = Q3 + (factor * IQR)
        
        # Znajdujemy indeksy wierszy, które mieszczą się w normie
        filter_mask = (X_temp[col] >= lower_bound) & (X_temp[col] <= upper_bound)
        valid_indices = valid_indices.intersection(X_temp[filter_mask].index)

    # Zwracamy przefiltrowane dane
    return X_temp.loc[valid_indices], y_temp.loc[valid_indices]

# Wybór kolumn do czyszczenia. 
# 'capital-gain' i 'capital-loss' celowo pominięte, bo tam wysokie wartości są kluczowe dla predykcji >50K
cols_to_check = ['age', 'hours-per-week', 'education-num']

print(f"Liczba wierszy przed usunięciem outlierów: {len(df_X_cleaned)}")

# Tworzymy nowe zmienne
df_X_final, df_y_final = remove_outliers_iqr(df_X_cleaned, df_y, cols_to_check)

print(f"Liczba wierszy po usunięciu outlierów: {len(df_X_final)}")

Przygotowabo dane do analizy eksploracyjnej, łącząc cechy z etykietami i zamieniając wartości binarne na czytelne nazwy kategorii, co ułatwia interpretację wyników

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ustawienia wykresów
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Łączymy X i y w jeden DataFrame tymczasowo do celów wizualizacji
df_eda = df_X_final.copy()

df_eda['income'] = df_y_final['income']  # 0 to <=50K, 1 to >50K

# Zamieniamy 0/1 z powrotem na etykiety dla czytelności wykresów
df_eda['income_label'] = df_eda['income'].map({0: '<=50K', 1: '>50K'})

df_eda.head()

In [ ]:
# Wykres zmiennej celu
plt.figure(figsize=(8, 5))
ax = sns.countplot(x='income_label', data=df_eda, palette='viridis', order=['<=50K', '>50K'], hue='income_label', legend=False)

# Dodanie liczebności nad słupkami
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + 0.35, p.get_height() + 100))

plt.title('Rozkład zmiennej celu (Income)', fontsize=15)
plt.xlabel('Dochód roczny')
plt.ylabel('Liczba osób')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='income_label', y='age', data=df_eda, palette='coolwarm', order=['<=50K', '>50K'], hue='income_label', legend=False)
plt.title('Rozkład wieku względem dochodu', fontsize=15)
plt.xlabel('Dochód')
plt.ylabel('Wiek')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

ax = sns.histplot(
    data=df_eda,
    x='education-num',
    hue='income_label',
    multiple='fill',
    bins=16,
    palette='magma',
    legend=True
)

plt.title('Procentowy udział grup dochodowych w zależności od lat edukacji', fontsize=15)
plt.xlabel('Liczba lat edukacji (education-num)')
plt.ylabel('Udział procentowy')

# Poprawne wyświetlenie legendy
sns.move_legend(ax, "upper right", title="Income")

plt.show()


In [ ]:
# Wybieramy tylko kolumny numeryczne do korelacji
numeric_cols = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income']
corr_matrix = df_eda[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Macierz korelacji zmiennych numerycznych', fontsize=15)
plt.show()

Ta macierz korelacji pokazuje, że najsilniejszy (choć wciąż umiarkowany) związek z dochodem ma poziom wykształcenia (0.33) oraz wiek (0.26). Pozostałe zmienne są ze sobą skorelowane bardzo słabo, co sugeruje brak silnych zależności liniowych między nimi w tym zbiorze.

Przeprowadzono analizę skupień przy użyciu algorytmu KMeans, skupiając się na kluczowych zmiennych takich jak wiek czy wykształcenie. Zastosowano metodę łokcia z automatycznym wyznaczaniem punktu przegięcia, co pozwoliło na optymalne pogrupowanie obserwacji.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Do klasteryzacji użyjemy tylko głównych zmiennych numerycznych
cols_for_clustering = ['age', 'education-num', 'hours-per-week', 'capital-gain']

# Skalowanie
scaler_cluster = StandardScaler()
X_cluster = scaler_cluster.fit_transform(df_eda[cols_for_clustering])

# Metoda łokcia
inertia = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_cluster)
    inertia.append(kmeans.inertia_)

# Automatyczne wykrycie łokcia
K = np.array(list(K_range))
inertia_arr = np.array(inertia)

# Linia łącząca pierwszy i ostatni punkt
p1 = np.array([K[0], inertia_arr[0]])
p2 = np.array([K[-1], inertia_arr[-1]])

# Odległość punktów od tej linii
distances = []
for i in range(len(K)):
    p = np.array([K[i], inertia_arr[i]])
    distance = np.abs(np.cross(p2 - p1, p1 - p)) / np.linalg.norm(p2 - p1)
    distances.append(distance)

elbow_k = K[np.argmax(distances)]
elbow_inertia = inertia_arr[np.argmax(distances)]

plt.figure(figsize=(10, 6))
plt.plot(K, inertia_arr, marker='o', linestyle='--', label='Inertia')
plt.scatter(elbow_k, elbow_inertia, color='red', s=120, label=f'Łokieć: k={elbow_k}')
plt.axvline(x=elbow_k, color='red', linestyle=':', alpha=0.7)

plt.title('Metoda łokcia – automatyczny wybór liczby klastrów')
plt.xlabel('Liczba klastrów (k)')
plt.ylabel('Suma kwadratów odległości wewnątrz klastra (Inertia)')
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
# Ustawiamy liczbę klastrów
optimal_k = 5

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df_eda['cluster'] = kmeans.fit_predict(X_cluster)

# Średnie wartości cech w każdym klastrze
cluster_summary = df_eda.groupby('cluster')[cols_for_clustering + ['income']].mean()
cluster_counts = df_eda['cluster'].value_counts().sort_index()

print("--- Charakterystyka klastrów (średnie wartości) ---")
print(cluster_summary)
print("\n--- Liczebność klastrów ---")
print(cluster_counts)

# Wizualizacja klastrów
plt.figure(figsize=(10, 6))
sns.scatterplot(x='age', y='hours-per-week', hue='cluster', data=df_eda, palette='bright', alpha=0.6)
plt.title('Segmentacja: Wiek vs Godziny pracy (kolory to klastry)')
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    df_eda['age'],
    df_eda['hours-per-week'],
    df_eda['capital-gain'],
    c=df_eda['cluster'],
    cmap='tab10',
    alpha=0.6
)

ax.set_xlabel('Age')
ax.set_ylabel('Hours per week')
ax.set_zlabel('Capital gain')
ax.set_title('Klastry w przestrzeni 3D')

legend = ax.legend(*scatter.legend_elements(), title="Cluster")
ax.add_artist(legend)

plt.show()


### 3. Graficzna i opisowa analiza eksploracyjna (EDA)

#### 3.1. Analiza zmiennej celu i korelacji
* **Niezbalansowanie klas:**
    * Po oczyszczeniu danych z wartości odstających (outlierów), proporcje klas pozostają zbliżone do pierwotnych: większość stanowią osoby o dochodzie **<=50K**, co narzuca konieczność stosowania metryki F1-score podczas oceny modeli.
* **Kluczowe korelacje:**
    1.  **Education-num:** Nadal pozostaje silnym predyktorem – wyższy poziom edukacji wyraźnie zwiększa szansę na wysokie zarobki.
    2.  **Age:** Zarobki rosną wraz z doświadczeniem, choć po osiągnięciu wieku emerytalnego (widoczne w klastrach starszych) tendencja ta może wyhamowywać.
    3.  **Hours-per-week:** Istotna korelacja dodatnia, co potwierdziła analiza klastrów (szczególnie w grupie o średnim wykształceniu).

#### 3.2. Wyniki segmentacji (Algorytm K-Means)
Na podstawie przetworzonych danych wyłoniono **5 charakterystycznych klastrów**, które ujawniają różne ścieżki do osiągnięcia wyższych dochodów:

* **Klaster 0: "Start kariery / Niskie kwalifikacje" (~36% populacji)**
    * **Liczebność:** 12 285 osób (najliczniejsza grupa).
    * **Profil:** Najmłodsza grupa (śr. 29 lat) ze średnim wykształceniem (~9 lat) i standardowym czasem pracy.
    * **Dochód:** Tylko **8.9%** zarabia >50K. Jest to grupa bazowa, dopiero wchodząca na rynek pracy lub wykonująca proste zawody.

* **Klaster 4: "Starsi z niższym wykształceniem" (~23.5% populacji)**
    * **Liczebność:** 7 990 osób.
    * **Profil:** Najstarsza grupa (śr. 52 lata), ale o poziomie edukacji zbliżonym do grupy najmłodszej (~9 lat).
    * **Dochód:** **25.5%** zarabia >50K.
    * *Wniosek:* Wiek i doświadczenie rekompensują braki w edukacji tylko w umiarkowanym stopniu.

* **Klaster 3: "Wykształceni profesjonaliści" (~22.5% populacji)**
    * **Liczebność:** 7 671 osób.
    * **Profil:** Osoby w średnim wieku (~39 lat) z **najwyższym poziomem edukacji** (śr. 13.2 lat, co odpowiada studiom wyższym) pracujące standardowe 40h/tydzień.
    * **Dochód:** **41.4%** zarabia >50K.
    * *Wniosek:* Wysokie kwalifikacje pozwalają na wysokie zarobki bez konieczności pracy w nadgodzinach.

* **Klaster 2: "Pracowici średniego szczebla" (~17% populacji)**
    * **Liczebność:** 5 912 osób.
    * **Profil:** Podobny wiek co w Klastrze 3 (~39 lat) i średnie wykształcenie (~11 lat), ale wyróżniający się **bardzo wysokim nakładem pracy** (śr. 49h/tydz.).
    * **Dochód:** **41.1%** zarabia >50K.
    * *Wniosek:* Osoby z niższym wykształceniem niż profesjonaliści "nadrabiają" zarobki, pracując znacznie ciężej (nadgodziny).

* **Klaster 1: "Zamożni inwestorzy" (<0.5% populacji)**
    * **Liczebność:** 142 osoby (grupa elitarna).
    * **Profil:** Grupa specyficzna, zdefiniowana przez maksymalny zysk kapitałowy (**$99,999**).
    * **Dochód:** **100%** osób w tej grupie zarabia >50K. Są to outliery pod względem finansowym, które celowo pozostawiono w zbiorze danych ze względu na ich istotność predykcyjną.

Następnie, uzyto One-Hot Encoding. Dodatkowo użyto StandardScaler dla zmiennych liczbowych, żeby takie wartości jak wiek czy godziny pracy miały podobną skalę i nie dominowały nad resztą cech podczas uczenia.

In [ ]:
# One-Hot Encoding dla zmiennych kategorycznych
categorical_cols = df_X_final.select_dtypes(include=['object']).columns

# Używamy get_dummies (pandas) dla szybkiego kodowania
df_X_encoded = pd.get_dummies(df_X_final, columns=categorical_cols, drop_first=True)

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(
    df_X_encoded, 
    df_y_final['income'], 
    test_size=0.2, 
    random_state=42, 
    stratify=df_y_final['income']
)

# Standaryzacja zmiennych numerycznych
# Skalujemy tylko na podstawie zbioru treningowego, żeby nie było wycieku danych
scaler = StandardScaler()

# Lista kolumn numerycznych, które zostały w danych
num_cols_to_scale = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

X_train[num_cols_to_scale] = scaler.fit_transform(X_train[num_cols_to_scale])
X_test[num_cols_to_scale] = scaler.transform(X_test[num_cols_to_scale])

print("Gotowe wymiary danych treningowych:", X_train.shape)
print("Przykładowe 5 wierszy danych po przetworzeniu:")
X_train.head()

**Regresja logistyczna:**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_curve, auc, precision_recall_curve, 
                             accuracy_score, f1_score)

# Model bazowy
lr_model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)

# Siatka hiperparametrów
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

# Tuning przy użyciu GridSearchCV (8-krotna walidacja krzyżowa, metryka f1)
grid_search = GridSearchCV(
    estimator=lr_model,
    param_grid=param_grid,
    cv=8,
    scoring='f1',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_lr = grid_search.best_estimator_
print(f"Najlepsze parametry: {grid_search.best_params_}")
print(f"Najlepszy wynik F1: {grid_search.best_score_:.4f}")

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    print(classification_report(y_test, y_pred))
    
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['<=50K', '>50K'], 
                yticklabels=['<=50K', '>50K'])
    plt.title('Macierz Konfuzji')
    plt.ylabel('Rzeczywistość')
    plt.xlabel('Predykcja')
    plt.savefig('confusion_matrix.png')
    
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Krzywa ROC (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Krzywa ROC')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.savefig('roc_curve.png')
    
    plt.show()

# Wywołanie ewaluacji (po wytrenowaniu)
evaluate_model(best_lr, X_test, y_test)

**Optymalne parametry:** Najlepsze wyniki (F1-score $\approx$ 0,66) model osiągnął przy silnej regularyzacji L1 (C=100), co sugeruje, że tylko część cech ma kluczowy wpływ na wynik.

**Problem czułości (Recall):** Model poprawnie identyfikuje 93% osób zarabiających $\le 50K$, ale ma trudności z wykrywaniem klasy mniejszościowej – tylko 60% osób z wysokim dochodem zostało poprawnie sklasyfikowanych.

**Ogólna skuteczność:** Wysoka dokładność (Accuracy = 84%) jest nieco myląca ze względu na niezbalansowanie klas; model jest "ostrożny" w przypisywaniu wysokich zarobków, co potwierdza przewaga precyzji (0,76) nad czułością (0,60) dla klasy 1.

In [ ]:
feature_names = X_train.columns
coefficients = best_lr.coef_[0]

feat_importances = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
feat_importances['AbsCoefficient'] = feat_importances['Coefficient'].abs()
feat_importances = feat_importances.sort_values(by='Coefficient', ascending=False)

# Wizualizacja 10 najsilniejszych pozytywnych i 10 najsilniejszych negatywnych wpływów
top_bottom_features = pd.concat([feat_importances.head(10), feat_importances.tail(10)])

plt.figure(figsize=(10, 8))
sns.barplot(x='Coefficient', y='Feature', data=top_bottom_features)
plt.title('Najważniejsze cechy w modelu Regresji Logistycznej', fontsize=15)
plt.xlabel('Wartość współczynnika (Wpływ na prawdopodobieństwo >50K)')
plt.show()

**GradientBoosting:**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

gb_model = GradientBoostingClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200],        # Liczba drzew
    'learning_rate': [0.05, 0.1, 0.2], # Jak szybko model się uczy
    'max_depth': [3, 4, 5],            # Głębokość pojedynczego drzewa
    'subsample': [0.8, 1.0]            # Użycie części danych do każdego drzewa
}

# Konfiguracja przeszukiwania siatki (Grid Search) z walidacją krzyżową (CV)
grid_search = GridSearchCV(
    estimator=gb_model,
    param_grid=param_grid,
    cv=3,                 # 3-krotna walidacja krzyżowa
    scoring='f1',         # Optymalizacja pod kątem F1-score (dla klasy 1)
    n_jobs=-1,
    verbose=1
)

# Trenowanie modelu
grid_search.fit(X_train, y_train)

# Wyniki tuningu
best_gb_model = grid_search.best_estimator_

print("\n--- WYNIKI TUNINGU ---")
print(f"Najlepsze parametry: {grid_search.best_params_}")
print(f"Najlepszy wynik F1-Score (cross-validation): {grid_search.best_score_:.4f}")

# Predykcja na zbiorze testowym
y_pred = best_gb_model.predict(X_test)

# Wyświetlenie raportu klasyfikacji
print("\n--- RAPORT KLASYFIKACJI (ZBIÓR TESTOWY) ---")
print(classification_report(y_test, y_pred))

# Wyświetlenie Macierzy Pomyłek
print("--- MACIERZ POMYŁEK ---")
print(confusion_matrix(y_test, y_pred))

F1-Score = 0.7076. Gradient Boosting okazał się bardziej skuteczny od regresji logistycznej

Accuracy = 87%: Model ogólnie myli się rzadko.

Recall dla klasy 1 (0.66).
Model rzadziej pomija osoby zamożne przy zachowaniu wysokiej precyzji (0,79)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

fig, ax = plt.subplots(figsize=(8, 6))

ConfusionMatrixDisplay.from_estimator(
    best_gb_model, 
    X_test, 
    y_test, 
    display_labels=['<=50K', '>50K'],
    cmap='Blues',
    normalize=None,
    ax=ax 
)

ax.set_title("Macierz Pomyłek (Liczebność)")
plt.show()

In [ ]:
from sklearn.metrics import RocCurveDisplay

fig, ax = plt.subplots(figsize=(8, 6))

RocCurveDisplay.from_estimator(
    best_gb_model, 
    X_test, 
    y_test, 
    name="Gradient Boosting",
    ax=ax
)

ax.plot([0, 1], [0, 1], "k--", label="AUC = 0.5")
ax.set_title("Krzywa ROC")
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

importances = best_gb_model.feature_importances_
feature_names = X_train.columns
feat_importances = pd.Series(importances, index=feature_names)

feat_importances.nlargest(10).plot(kind='barh', ax=ax, color='teal', edgecolor='black')

ax.set_title("TOP 10 Cech decydujących o dochodzie")
ax.set_xlabel("Waga cechy (Importance)")
ax.invert_yaxis()

plt.show()

**Random Forest:**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definicja modelu
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Siatka parametrów do przetestowania
# Sprawdzamy liczbę drzew, głębokość oraz wagę klas
param_grid_rf = {
    'n_estimators': [100, 200],       # Liczba drzew
    'max_depth': [10, 20, None],      # Maksymalna głębokość
    'min_samples_leaf': [1, 4],       # Minimalna liczba próbek w liściu
    'class_weight': ['balanced', None] # Czy wyrównywać wagi dla mniejszej klasy (>50K)
}

# Konfiguracja przeszukiwania (optymalizacja pod kątem F1-score)
grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    cv=8,             # 8-krotna walidacja krzyżowa
    scoring='f1',     # Skupiamy się na F1 
    verbose=1,
    n_jobs=-1
)

grid_search_rf.fit(X_train, y_train)

best_rf = grid_search_rf.best_estimator_
print(f"\nNajlepsze parametry: {grid_search_rf.best_params_}")
print(f"Najlepszy wynik F1 (CV): {grid_search_rf.best_score_:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, RocCurveDisplay

y_pred_rf = best_rf.predict(X_test)

# Raport tekstowy
print(classification_report(y_test, y_pred_rf, target_names=['<=50K', '>50K']))

# Ustawienie obszaru wykresów
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Macierz Pomyłek (Znormalizowana)
ConfusionMatrixDisplay.from_estimator(best_rf, X_test, y_test, 
                                      display_labels=['<=50K', '>50K'], 
                                      cmap='Greens', normalize='true', ax=axes[0])
axes[0].set_title('Znormalizowana Macierz Pomyłek')

# Krzywa ROC
RocCurveDisplay.from_estimator(best_rf, X_test, y_test, ax=axes[1], name='Random Forest')
axes[1].plot([0, 1], [0, 1], 'k--', label='Losowy wybór')
axes[1].set_title('Krzywa ROC')

# Ważność cech (Feature Importance)h
importances = best_rf.feature_importances_
# Pobranie nazw kolumn
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f"Feature {i}" for i in range(X_train.shape[1])]
feat_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False).head(10)

sns.barplot(x=feat_importances.values, y=feat_importances.index, palette='viridis', ax=axes[2], hue=feat_importances.index, legend=False)
axes[2].set_title('TOP 10 Najważniejszych Cech')
axes[2].set_xlabel('Waga cechy (Gini Importance)')

plt.tight_layout()
plt.show()

* **Wysoka Czułość (Recall) dla klasy >50K: 80%**
    * Model skutecznie "wyłapuje" osoby zamożne.
    * Bardzo niskie ryzyko przeoczenia klienta docelowego.

* **Precyzja (Precision) dla klasy >50K: 62%**
    * Koszt wysokiej czułości: spora liczba False Positives.
    * Model jest nadgorliwy – czasem klasyfikuje klasę średnią jako bogatą.

* **Accuracy: 82%**
    * Wynik niższy od modeli prostych, ale moze byc bardziej wartościowy biznesowo (model nie ignoruje mniejszości).

**Wniosek:**
Model przyjmuje strategię agresywną. Jest dobry, gdy koszt pominięcia zamoznego 
klienta jest wyższy niż koszt pomyłki.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, f1_score, accuracy_score
import seaborn as sns
import pandas as pd

models = {
    "Logistic Regression": (best_lr, X_test),
    "Gradient Boosting": (best_gb_model, X_test),
    "Random Forest": (best_rf, X_test)
}

# 2. Wspólny wykres krzywych ROC
plt.figure(figsize=(10, 8))
for name, (model, data_test) in models.items():
    probs = model.predict_proba(data_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--', label='Losowy wybór')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Porównanie krzywych ROC')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()

# 3. Porównanie macierzy
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, (name, (model, data_test)) in enumerate(models.items()):
    y_pred = model.predict(data_test)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i], cbar=False)
    axes[i].set_title(f'Macierz: {name}')
    axes[i].set_xticklabels(['<=50K', '>50K'])
    axes[i].set_yticklabels(['<=50K', '>50K'])

plt.tight_layout()
plt.show()

# Wybór najlepszego modelu
comparison_data = []
for name, (model, data_test) in models.items():
    y_pred = model.predict(data_test)
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    comparison_data.append({"Model": name, "F1-Score": f1, "Accuracy": acc})

df_comp = pd.DataFrame(comparison_data).sort_values(by="F1-Score", ascending=False)
print("\nRANKING MODELI (wg F1-Score):")
print(df_comp.to_string(index=False))

best_model_name = df_comp.iloc[0]['Model']
print(f"\nNAJLEPSZY MODEL: {best_model_name}")

W ostatnim etapie prac zestawiono wszystkie wytrenowane modele, aby bezpośrednio porównać ich skuteczność na zbiorze testowym przy użyciu wspólnej charakterystyki ROC oraz macierzy pomyłek. Wygenerowanie zbiorczego rankingu pozwala na ocenę, który algorytm najlepiej radzi sobie z wyważeniem precyzji i czułości w problemie klasyfikacji dochodów. Dzięki wizualizacji krzywych ROC mogliśmy sprawdzić ogólną zdolność modeli do odseparowania obu klas niezależnie od przyjętego progu prawdopodobieństwa.

**Zwycięzca zestawienia:** Najlepsze wyniki osiągnął model **Gradient Boosting**, który uzyskał najwyższy wskaźnik F1-Score **(0,719)** oraz najwyższą ogólną dokładność (86,6%), co czyni go najbardziej zbalansowanym klasyfikatorem.

**Charakterystyka Random Forest:** Model ten zajął drugie miejsce w rankingu; mimo bardzo wysokiej zdolności do wykrywania osób zamożnych (wysoki recall), generował on więcej błędów typu False Positive niż Gradient Boosting.

Wszystkie modele uzyskały **wysokie i zbliżone wartości AUC**, co potwierdza poprawność przeprowadzonego procesu inżynierii cech i przygotowania danych.